In [53]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs import whichBisSig_NAHS, B_ID, ancestor_below_B

In [54]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-10-21 14:48:50.255413


In [55]:
save_data = False
save_preprocessedDataframe = True
tmp_data = False
take_subset = False
subset_size = 100000

delete_top_numFSP_evts=False

In [56]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "FULLCMrun_Dstlnu_SHR_BsX_Ablations/"
root_subdir = "SHR_CMvars_run1/"   # "SHR_LeasVars_run2/"

root_path = nfs_path + "SHR_Hc_correctReco_BsX/" + root_subdir # + sub + "/"
root_path = "/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/"

In [57]:
merged = "merged_"
if tmp_data:
    merged += "tmp_"
merged = ""

In [58]:
fileHc = uproot.open(root_path + merged + "allHc.root:variables")

In [59]:
df_Hc = fileHc.arrays(library="pd")

In [60]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename=root_path + merged + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/gammas.root:variables
/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/electrons.root:variables
/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/pions.root:variables
/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/kaons.root:variables
/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/muons.root:variables


In [61]:
df_FSPs = pd.concat(dfs)

In [62]:
# delete FSPs for which no Y4S file entry was found
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [63]:
df_FSPs.keys()

Index(['__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'genMothPDG_0', 'genMothPDG_1',
       'genMothPDG_2', 'genMothPDG_3', 'genMothPDG_4', 'genMothPDG_5',
       'genMothPDG_6', 'genMothPDG_7', 'genMothPDG_8', 'genMothPDG_9',
       'genMotherID_0', 'genMotherID_1', 'genMotherID_2', 'genMotherID_3',
       'genMotherID_4', 'genMotherID_5', 'genMotherID_6', 'genMotherID_7',
       'genMotherID_8', 'genMotherID_9', 'mcMother0_uniqParID',
       'mcMother1_uniqParID', 'mcMother2_uniqParID', 'mcMother3_uniqParID',
       'mcMother4_uniqParID', 'mcMother5_uniqParID', 'mcMother6_uniqParID',
       'mcMother7_uniqParID', 'mcMother8_uniqParID', 'mcMother9_uniqParID',
       'cmpx', 'cmpy', 'cmpz', 'cmE', 'isSignal', 'uniqueParticleIdentifier',
       'mcErrors', 'mcPDG', 'genMotherID', 'genMotherP', 'genMotherPDG',
       'charge', 'dr', 'dz', 'clusterReg', 'clusterE9E21', 'M', 'PDG',
       'genParticleID', 'px', 'py', 'pz', 'pt', 'p', 'E'

## check that each particles occurs only ones

In [64]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
groupsFSPs_uniqParID.value_counts('count')

count
1    2991
dtype: int64

## add cols

In [65]:
df_Hc['Bsig_uniqParID'] = df_Hc.apply(whichBisSig_NAHS, axis=1)

In [66]:
df_Hc['Bsig_uniqParID'].value_counts()

83886082.0    95
83886081.0    83
Name: Bsig_uniqParID, dtype: int64

In [67]:
# function to create col with the particles mother B's uniqueParticleIdentifier
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

In [68]:
df_FSPs['B_ID'].value_counts()

83886081    1349
83886082    1255
0            387
Name: B_ID, dtype: int64

## double between gamma and charged?

In [69]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
groupsFSPs_uniqParID["count"].max()

1

## check D*lnu

In [70]:
groupsHc = pd.DataFrame({'count' : df_Hc[df_Hc["Bsig_uniqParID"] == 83886082.0].groupby( 
    ["genUp4S_PDG_1_0","genUp4S_PDG_1_1","genUp4S_PDG_1_2"] ).size()}).reset_index()

groupsHc

,genUp4S_PDG_1_0,genUp4S_PDG_1_1,genUp4S_PDG_1_2,count
0,-413.0,-13.0,14.0,22
1,-413.0,-11.0,12.0,27
2,413.0,11.0,-12.0,22
3,413.0,13.0,-14.0,24


In [71]:
groupsHc = pd.DataFrame({'count' : df_Hc[df_Hc["Bsig_uniqParID"] == 83886081.0].groupby( 
    ["genUp4S_PDG_0_0","genUp4S_PDG_0_1","genUp4S_PDG_0_2"] ).size()}).reset_index()

groupsHc

,genUp4S_PDG_0_0,genUp4S_PDG_0_1,genUp4S_PDG_0_2,count
0,-413.0,-13.0,14.0,21
1,-413.0,-11.0,12.0,18
2,413.0,11.0,-12.0,24
3,413.0,13.0,-14.0,20


## delete evts where Hc mother and Bsig overlap

In [72]:
print(df_Hc.shape[0])

178


In [73]:
df_Hc[df_Hc['Bsig_uniqParID'] == df_Hc['mcMother0_uniqParID']][["__event__","uniqueParticleIdentifier","PDG",
    "genMothPDG_0",'Bsig_uniqParID',"mcMother0_uniqParID"]][:10]

,__event__,uniqueParticleIdentifier,PDG,genMothPDG_0,Bsig_uniqParID,mcMother0_uniqParID
51,2331013,100663296.0,-413.0,511.0,83886081.0,83886081.0
52,2331096,100663296.0,-413.0,511.0,83886082.0,83886082.0
53,2331142,100663296.0,413.0,-511.0,83886081.0,83886081.0
54,2331249,100663296.0,-413.0,511.0,83886082.0,83886082.0
55,2331349,100663296.0,413.0,-511.0,83886082.0,83886082.0
56,2331398,100663296.0,413.0,-511.0,83886081.0,83886081.0
57,2331398,100663296.0,413.0,-511.0,83886081.0,83886081.0
61,2331499,100663296.0,-413.0,511.0,83886082.0,83886082.0
62,2331581,100663296.0,413.0,-511.0,83886082.0,83886082.0
63,2331619,100663296.0,413.0,-511.0,83886081.0,83886081.0


In [74]:
df_Hc = df_Hc[df_Hc['Bsig_uniqParID'] != df_Hc['mcMother0_uniqParID']]

In [75]:
print(df_Hc.shape[0])

97


In [76]:
#[["__event__","uniqueParticleIdentifier","PDG","genMothPDG_0",'Bsig_uniqParID',"mcMother0_uniqParID"]]

## take care of duplis in df_Hc, delete them, because with the prior, duplis here mean there must have been an error

In [77]:
groupsHc_uniqParID = pd.DataFrame({'count' : df_Hc.groupby( ["__event__"] ).size()}).reset_index()
groupsHc_uniqParID.value_counts('count')

count
1    95
2     1
dtype: int64

In [78]:
df_Hc[df_Hc['__event__'].isin(groupsHc_uniqParID[groupsHc_uniqParID["count"]>1]["__event__"])][["__event__",
    "uniqueParticleIdentifier","PDG",
    "genMothPDG_0",'Bsig_uniqParID',"mcMother0_uniqParID","px","E","x","genUp4S_PDG_1_0"]][:10]

,__event__,uniqueParticleIdentifier,PDG,genMothPDG_0,Bsig_uniqParID,mcMother0_uniqParID,px,E,x,genUp4S_PDG_1_0
20,2332070,100663296.0,-411.0,-511.0,83886081.0,83886082.0,-0.257812,2.284948,-0.051214,411.0
21,2332070,100663296.0,411.0,-511.0,83886081.0,83886082.0,0.036926,2.162483,-0.044568,411.0


In [79]:
df_Hc = df_Hc[df_Hc['__event__'].isin(groupsHc_uniqParID[groupsHc_uniqParID["count"]==1]["__event__"])]

In [80]:
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [81]:
groupsHc_uniqParID = pd.DataFrame({'count' : df_Hc.groupby( ["__event__"] ).size()}).reset_index()
groupsHc_uniqParID.value_counts('count')

count
1    95
dtype: int64

## delete the H_c FSPs 

In [82]:
df_FSPs['ancestor_below_B_arrayID'] = df_FSPs.apply(ancestor_below_B, axis=1)

In [83]:
df_Hc["HcMother_uniqParID"] = df_Hc["mcMother0_uniqParID"]
df_Hc["Hc_genParticleID"] = df_Hc["genParticleID"]
df_Hc["Hc_MotherPDG"] = df_Hc["genMothPDG_0"]

In [84]:
df_FSPs = pd.merge(df_FSPs,df_Hc[["__event__","Bsig_uniqParID","HcMother_uniqParID","Hc_genParticleID","Hc_MotherPDG"]], on="__event__")

In [85]:
df_FSPs[["__event__","Bsig_uniqParID","HcMother_uniqParID","Hc_genParticleID","ancestor_below_B_arrayID","Hc_MotherPDG"]]

,__event__,Bsig_uniqParID,HcMother_uniqParID,Hc_genParticleID,ancestor_below_B_arrayID,Hc_MotherPDG
0,2331192,83886082.0,83886081.0,3.0,0.0,511.0
1,2331192,83886082.0,83886081.0,3.0,0.0,511.0
2,2331192,83886082.0,83886081.0,3.0,6.0,511.0
3,2331192,83886082.0,83886081.0,3.0,4.0,511.0
4,2331192,83886082.0,83886081.0,3.0,6.0,511.0
...,...,...,...,...,...,...
1677,2335954,83886081.0,83886082.0,7.0,6.0,511.0
1678,2335954,83886081.0,83886082.0,7.0,7.0,511.0
1679,2335954,83886081.0,83886082.0,7.0,-1.0,511.0
1680,2335954,83886081.0,83886082.0,7.0,7.0,511.0


In [86]:
df_FSPs.shape[0]

1682

In [87]:
# deletion of Hc FSPs, those who 
df_FSPs = df_FSPs[~((df_FSPs["Hc_genParticleID"] == df_FSPs["ancestor_below_B_arrayID"]) &  # originate from Hc PDG
                  (df_FSPs["HcMother_uniqParID"] == df_FSPs["B_ID"]))] # and are B-tag ancestors

In [88]:
df_FSPs.shape[0]

1276

In [89]:
#keep[keep["HcMother_uniqParID"]==keep["B_ID"]][["__event__","Bsig_uniqParID","HcMother_uniqParID","Hc_PDG","ancestor_below_B","B_ID","Hc_MotherPDG"]]

In [90]:
#df_Hc[df_Hc["__event__"] == 2331192][["genUp4S_PDG_0_0","genUp4S_PDG_0_1","genUp4S_PDG_0_2"]]

## delete delete_top_numFSP_evts

In [91]:
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
numEvtsBeforeCut = df_Hc.shape[0]

In [92]:
# print first to adjust percentile used
print("original max FSPs:",numberFSPsEvts["count"].max())
for percentile_lvl in [0.995,0.996,0.997,0.998,0.999]:
    print(percentile_lvl)
    percentile = numberFSPsEvts["count"].quantile(q=percentile_lvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    #TMP_df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    TMP_df_Hc = df_Hc[df_Hc['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
    print("would delete", round(100 - (TMP_df_Hc.shape[0]/df_Hc.shape[0])*100,2),"% of evts")
    print("new max FSPs:",cut_numberFSPsEvts["count"].max())

original max FSPs: 22
0.995
would delete 1.05 % of evts
new max FSPs: 21
0.996
would delete 1.05 % of evts
new max FSPs: 21
0.997
would delete 1.05 % of evts
new max FSPs: 21
0.998
would delete 1.05 % of evts
new max FSPs: 21
0.999
would delete 1.05 % of evts
new max FSPs: 21


In [93]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])


if delete_top_numFSP_evts:
    delete_percentileLvl = 0.998
    print("used delete_percentileLvl:",delete_percentileLvl)
    
    
    print(numberFSPsEvts["count"].describe())
    
    percentile = numberFSPsEvts["count"].quantile(q=delete_percentileLvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    df_Hc = df_Hc[df_Hc['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
print(df_FSPs.shape[0])
print(df_Hc.shape[0])
print("deleted", round(100 - (df_Hc.shape[0]//numEvtsBeforeCut)*100,2),"% of events")
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
print(numberFSPsEvts["count"].describe())

1276
95
1276
95
deleted 0 % of events
count    95.000000
mean     13.431579
std       3.822255
min       5.000000
25%      11.000000
50%      13.000000
75%      16.000000
max      22.000000
Name: count, dtype: float64


## impute the nan values of variables

In [94]:
## data preprocessing, dealing with NaN
imputelist = [["pionID",-1.],["kaonID",-1.],["electronID",-1.],["muonID",-1.]
              ,["clusterReg",-1.],["clusterE9E21",-1.]
              ]
for impute in imputelist:
    column_name = impute[0]
    impute_val = impute[1]
    mask = df_FSPs[column_name].isna() == 1
    df_FSPs.loc[mask, column_name] = impute_val
    print(df_FSPs[df_FSPs[column_name].isna() == True].shape[0])

0
0
0
0
0
0


## last check if events match between dfs

In [95]:
df_Hc = df_Hc[df_Hc['__event__'].isin(df_FSPs["__event__"])]
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [96]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])

1276
95


## save the dfs as they will be used for data prod

In [35]:
df_FSPs_final = df_FSPs
df_Y4S_final = df_Hc

In [36]:
root_path

'/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/'

In [37]:
df_FSPs_final.to_csv(root_path + "final_df_FSPs_preProcessed_NAHS.csv")
df_Y4S_final.to_csv(root_path + "final_df_Y4S_preProcessed_NAHS.csv")

## beginning of data prod

In [38]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')
print("num Events:",numFSPs.shape[0],'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 9
maxFSPs: 25 

num Events: 95 



In [39]:
variable_combinations = [
    [[    "charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_M"],
    [["M",         "dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_charge"],
    [["M","charge",     "dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_dr"],
    [["M","charge","dr",     "clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_dz"],
    [["M","charge","dr","dz",             "clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_clReg"],
    [["M","charge","dr","dz","clusterReg",               "pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_E9E21"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21",         "kaonID","electronID","muonID","protonID"],"NAHSA_no_pID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID",         "electronID","muonID","protonID"],"NAHSA_no_kID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID",             "muonID","protonID"],"NAHSA_no_eID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID"         ,"protonID"],"NAHSA_no_muID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"           ],"NAHSA_no_proID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_allExtras"],
    [["M","charge","pionID","kaonID","electronID","muonID"],"NAHSA_IDchargeM"],
    [["M","charge"],"NAHSA_chargeM"],
    [["charge","pionID","kaonID","electronID","muonID"],"NAHSA_chargeIDs"],
    [["charge","pionID"],"NAHSA_chargePionID"],
    [[],"NAHSA_only4mom"],
    [["charge"],"NAHSA_charge"]
]

In [40]:
variable_combinations = [
    [[],"NAHSA_only4mom"]
]

In [41]:
for var_comb in variable_combinations:
    ablationSub = var_comb[1]
    data_dir = Path(nfs_path + "data/" + data_subdir + root_subdir + ablationSub + "/")    
    if save_data:
        data_dir.mkdir(parents=True, exist_ok=True)
    print("Will save data to:", data_dir,'is', save_data ,'\n')


    #minFSPs = 5
    for num_FSPs_toData in range(minFSPs, maxFSPs+1):
        df_num_subset = df_FSPs_final.copy()
        df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]


        numEvents = df_num_subset.__event__.nunique()
        print("numEvents:",numEvents)
        print("num_FSPs_toData:",num_FSPs_toData)  
        if numEvents == 0:
            print("skipped because empty \n")
            continue

        if numEvents < 10:
            print("skipped because <10 events \n")
            continue

        num_features = 4 + len(var_comb[0])
        leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
        SA_target =  np.zeros((numEvents, num_FSPs_toData))
        global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)

        event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
        #print("len(event_list):",len(event_list))
        for i in range(numEvents):

            event_iter = event_list[i]

            global_tag_masterInfo = "evt" + str(event_iter)
            global_tag[i,-1] = global_tag_masterInfo
            #print("global_tag[i,-1]:",global_tag[i,-1])
            #print("i:",i,"event_iter:",event_iter)

            event_df = df_num_subset[df_num_subset.__event__ == event_iter]

            for j in range(num_FSPs_toData):
                #print("numParticle:",j)
                particle = event_df.iloc[j]

                #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
                leaves[i,j,0] = particle["px"]
                leaves[i,j,1] = particle["py"]
                leaves[i,j,2] = particle["pz"]
                leaves[i,j,3] = particle["E"]
                
                for k in range(len(var_comb[0])):
                    leav_idx = k + 4
                    leaves[i,j,leav_idx] = particle[var_comb[0][k]]
                    



                global_tag_Info = str((particle["mcPDG"])) 
                global_tag[i,j] = global_tag_Info

                label = -10 # error code if assignment fails
                B_tag_uniqID = -10 # error code if assignment fails     

                B_sig_uniqID = df_Y4S_final[df_Y4S_final["__event__"] == event_iter].iloc[0]['Bsig_uniqParID']
                if B_sig_uniqID == 83886082.0:
                    B_tag_uniqID = 83886081.0
                elif B_sig_uniqID == 83886081.0:
                    B_tag_uniqID = 83886082.0

                if particle["B_ID"] == B_tag_uniqID:
                    label = 1 # particle belongs to X (MC truth)
                elif particle["B_ID"] == B_sig_uniqID:
                    label = 2 # particle belongs to Bsig (MC truth)
                elif particle["B_ID"] == 0:
                    label = 0 # background (MC truth)


                SA_target[i,j] = label

            del event_df


        # shuffle the data    
        for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
            perms = np.random.permutation(leaves.shape[1])

            leaves[idx,:] = leaves[idx,perms]
            SA_target[idx,:] = SA_target[idx,perms]
            global_tag[idx,0:-1] = global_tag[idx,perms]





        #print(global_tag)
        train_ratio = 0.8
        validation_ratio = 0.2
        #test_ratio = 0.06

        print("leaves.shape:",leaves.shape)
        print("SA_target.shape:",SA_target.shape)
        print("global_tag.shape:",global_tag.shape)


        print("leaves[0]:",leaves[0])
        print("SA_target[0]:",SA_target[0])
        print("global_tag[0]:",global_tag[0])

        x=leaves
        y=SA_target
        z=global_tag

        x_train, x_val, y_train, y_val, z_train, z_val = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

        if save_data==True:
            np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
            np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
            np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

            np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
            np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
            np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

            #np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
            #np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
            #np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)


        print("")
        #del df_num_subset


        del df_num_subset


Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/FULLCMrun_Dstlnu_SHR_BsX_Ablations/SHR_CMvars_run1/NAHSA_only4mom is False 

numEvents: 2
num_FSPs_toData: 9
skipped because <10 events 

numEvents: 2
num_FSPs_toData: 10
skipped because <10 events 

numEvents: 4
num_FSPs_toData: 11
skipped because <10 events 

numEvents: 2
num_FSPs_toData: 12
skipped because <10 events 

numEvents: 4
num_FSPs_toData: 13
skipped because <10 events 

numEvents: 9
num_FSPs_toData: 14
skipped because <10 events 

numEvents: 3
num_FSPs_toData: 15
skipped because <10 events 

numEvents: 8
num_FSPs_toData: 16
skipped because <10 events 

numEvents: 9
num_FSPs_toData: 17
skipped because <10 events 

numEvents: 11
num_FSPs_toData: 18
leaves.shape: (11, 18, 4)
SA_target.shape: (11, 18)
global_tag.shape: (11, 19)
leaves[0]: [[ 0.44409418  0.29826891  0.2467518   0.58912684]
 [-0.13767813 -0.67586303  0.2329686   0.87962346]
 [ 0.33268592  0.14421238  0.27081567  0.45256851]
 [ 0.

In [42]:
print("saving is done")
now = datetime.now()
print("time at end =", now)

saving is done
time at end = 2021-10-21 13:37:14.389956
